In [1]:
import re
from collections import Counter

import numpy as np
import pandas as pd
import spacy
import torch
from radgraph import RadGraph
from rich import print

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Spacy's English tokenizer
nlp = spacy.load("en_core_web_sm")

# Initialize RadGraph
radgraph = RadGraph()

DATA_DIR = "C:/Users/DryLab/Desktop/villa/data/open-i"

c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_type not provided, defaulting to radgraph-xl


c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release

In [2]:
df = pd.read_feather(
    "C:/Users/DryLab/Desktop/ViLLA/data/open-i/annotations_region.feather"
)

In [3]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...


In [4]:
text_path_list = df["text_filepath"].tolist()

In [5]:
def text_cleaning(text):
    # Lowercase the text
    # text = text.lower()

    # Remove special characters
    # text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Remove extra whitespaces
    # text = re.sub(r"\s+", " ", text)

    # Remove punctuation
    # text = re.sub(r"[^\w\s]", "", text)

    # Remove stopwords nltk
    # text = " ".join(
    #     [
    #         word
    #         for word in text.split()
    #         if word not in nltk.corpus.stopwords.words("english")
    #     ]
    # )

    doc = nlp(text)

    tokens = [token.text.lower() for token in doc if not token.is_stop]

    cleaned_text = " ".join(tokens)

    return cleaned_text

In [6]:
def filter_sentences_by_absent_tokens(radgraph_output):
    remaining_sentences = []

    radgraph_output = radgraph(radgraph_output)

    # Iterate over RadGraph output
    for _, description in radgraph_output.items():
        text = description.get("text", " ")
        entities = description.get("entities", {})

        # Step 1: Split the text into sentences
        doc = nlp(text)
        sentences = [sent.text.lower() for sent in doc.sents]

        # Step 2: Create a mapping of word indices to sentence indices
        word_to_sentence = {}
        current_word_idx = 0

        for i, sentence in enumerate(sentences):
            sentence_words = sentence.split()
            for _ in sentence_words:
                word_to_sentence[current_word_idx] = i
                current_word_idx += 1

        # Step 3: Create a flag to keep or discard each sentence
        sentence_flags = [True] * len(sentences)

        # Step 4: Iterate over entities and check for tokens labeled "Observation::definitely absent"
        for _, entity_info in entities.items():
            label = entity_info.get("label")
            start_ix = entity_info.get("start_ix")

            # Check if the entity is labeled as "Observation::definitely absent"
            if label == "Observation::definitely absent":
                # Step 5: Find which sentence contains the token based on start and end indices
                sentence_idx = word_to_sentence.get(start_ix, None)
                if sentence_idx is not None:
                    sentence_flags[sentence_idx] = False

        # Step 6: Collect the remaining sentences
        for i, sentence in enumerate(sentences):
            if sentence_flags[i]:
                if not re.match(r"^\d\s*\.$", sentence.strip()):
                    remaining_sentences.append(sentence)

        # Revert back to the original text
        remaining_sentences = " ".join(remaining_sentences)

    return remaining_sentences

In [7]:
df["text"] = df["text_filepath"].apply(lambda x: open(x, "r").read())

In [8]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,There is XXXX increased opacity within the rig...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,There is XXXX increased opacity within the rig...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Interstitial markings are diffusely prominent ...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Status post left mastectomy. Heart size normal...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Heart size and pulmonary vascularity appear wi...
...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Cardiomediastinal silhouette demonstrates norm...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Lungs are clear bilaterally. There is no focal...
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Lungs are clear bilaterally. There is no focal...
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Normal heart and mediastinum. Clear lungs. Tra...


In [9]:
df.text_filepath.iloc[2]

'C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i/data/ecgen-radiology-txt\\CXR1001_IM-0004-1001.txt'

In [10]:
import random

index = random.randint(0, len(df) - 1)

findings = df["text"].iloc[index]
annotations = radgraph(findings)
filtered_text = filter_sentences_by_absent_tokens(findings)
cleaned_text = text_cleaning(filtered_text)

print(f"Original Text: {findings}")
print(f"Preprocessed Text: {filtered_text}")
print(f"Cleaned Text: {cleaned_text}")

Original Text: No focal consolidation. No visualized pneumothorax. The heart size is normal. There are no large 
pleural effusions. 1. No acute cardiopulmonary findings.

Preprocessed Text: the heart size is normal .

Cleaned Text: heart size normal .

In [11]:
# Break down the text into sentences and filter out the sentences that contain tokens labeled as "Observation::definitely absent"
df["text"] = df["text"].apply(filter_sentences_by_absent_tokens)

In [12]:
print(df.text.iloc[0])

there is xxxx increased opacity within the right upper lobe with possible mass and associated area of atelectasis 
or focal consolidation . the cardiac silhouette is within normal limits . xxxx opacity in the left midlung 
overlying the posterior left 5th rib may represent focal airspace disease . increased opacity in the right upper 
lobe with xxxx associated atelectasis may represent focal consolidation or mass lesion with atelectasis . recommend
chest ct for further evaluation . xxxx opacity overlying the left 5th rib may represent focal airspace disease .

In [21]:
df["text"] = df["text"].apply(text_cleaning)

In [13]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...
...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...


In [14]:
# Boolean mask where text is a string more than 2 words
mask = df["text"].apply(lambda x: len(x.split()) > 2)

# Assign filtered values to df column
df["text"] = df["text"].where(mask)

In [15]:
df.dropna(inplace=True)

In [16]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...
...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...


In [17]:
index = 0
findings = df.iloc[index]["text"]
annotation = radgraph([findings])
print(f"Findings: {findings}")

Findings: there is xxxx increased opacity within the right upper lobe with possible mass and associated area of 
atelectasis or focal consolidation . the cardiac silhouette is within normal limits . xxxx opacity in the left 
midlung overlying the posterior left 5th rib may represent focal airspace disease . increased opacity in the right 
upper lobe with xxxx associated atelectasis may represent focal consolidation or mass lesion with atelectasis . 
recommend chest ct for further evaluation . xxxx opacity overlying the left 5th rib may represent focal airspace 
disease .

In [18]:
annotation

{'0': {'text': 'there is xxxx increased opacity within the right upper lobe with possible mass and associated area of atelectasis or focal consolidation . the cardiac silhouette is within normal limits . xxxx opacity in the left midlung overlying the posterior left 5th rib may represent focal airspace disease . increased opacity in the right upper lobe with xxxx associated atelectasis may represent focal consolidation or mass lesion with atelectasis . recommend chest ct for further evaluation . xxxx opacity overlying the left 5th rib may represent focal airspace disease .',
  'entities': {'1': {'tokens': 'xxxx',
    'label': 'Observation::definitely present',
    'start_ix': 2,
    'end_ix': 2,
    'relations': [['modify', '2'], ['modify', '3']]},
   '2': {'tokens': 'increased',
    'label': 'Observation::definitely present',
    'start_ix': 3,
    'end_ix': 3,
    'relations': [['modify', '3']]},
   '3': {'tokens': 'opacity',
    'label': 'Observation::definitely present',
    'start_

In [19]:
def extract_entities(textual_descriptions):
    extracted_entities = []

    textual_descriptions = radgraph(textual_descriptions)

    for _, description in textual_descriptions.items():
        # Extract entities using RadGraph
        entities = description.get("entities", {})

        # Filter entities that are 'Definitely Present'
        for _, entities_label in entities.items():
            label = entities_label.get("label")
            tokens = entities_label.get("tokens")
            if label == "Observation::definitely present":
                extracted_entities.append(tokens)

    return extracted_entities


textual_descriptions = findings
entities = extract_entities(textual_descriptions)
entities = np.unique(entities).tolist()

In [20]:
entities

['atelectasis', 'increased', 'limits', 'normal', 'opacity', 'xxxx']

In [21]:
def filter_nouns(entities):
    noun_entities = []

    for entity in entities:
        # Tokenize the entity using Spacy
        doc = nlp(entity)
        for token in doc:
            # Filter out tokens that are nouns
            if token.pos_ == "NOUN":
                # Append the noun to the list of noun entities
                noun_entities.append(token.text)

    return noun_entities


noun_entities = filter_nouns(entities)

In [22]:
noun_entities

['atelectasis', 'limits', 'opacity']

In [23]:
df["attributes"] = df["text"].apply(extract_entities)

In [24]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[xxxx, increased, opacity, normal, limits, xxx..."
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[xxxx, increased, opacity, normal, limits, xxx..."
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[markings, diffusely, prominent, normal, norma..."
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[post, mastectomy, normal, clear]"
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[normal, density, air, this, hiatal hernia, ca..."
...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[normal, tortuosity, atherosclerosis, disease]"
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[clear, normal, limits, unremarkable]"
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[clear, normal, limits, unremarkable]"
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[normal, clear, midline, radiopaque, foreign b..."


In [25]:
df["attributes"] = df["attributes"].apply(filter_nouns)

In [26]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[opacity, limits, opacity, opacity, atelectasi..."
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[opacity, limits, opacity, opacity, atelectasi..."
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[markings, fibrosis]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[post, mastectomy]"
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[density, air, calcification, opacity, osteope..."
...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[tortuosity, atherosclerosis, disease]"
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[midline, radiopaque, body]"


In [27]:
# Remove duplicate attributes
df["attributes"] = df["attributes"].apply(lambda x: np.unique(x).tolist())

In [28]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]"
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]"
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[fibrosis, markings]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[mastectomy, post]"
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[air, atelectasis, calcification, density, opa..."
...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[atherosclerosis, disease, tortuosity]"
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[body, midline, radiopaque]"


In [38]:
# # Remove row if no attributes are less than 3
# mask = df["attributes"].apply(lambda x: len(x) > 0)

# df["attributes"] = df["attributes"].where(mask)

In [42]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]"
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]"
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[fibrosis, markings]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[mastectomy, post]"
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[air, atelectasis, calcification, density, opa..."
...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[atherosclerosis, disease, tortuosity]"
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits]
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[body, midline, radiopaque]"


In [43]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets (70% training and 30% test)
train_df, split_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

# Split the remaining data into validation and test sets (10% validation and 20% test)
val_df, test_df = train_test_split(split_df, test_size=2/3, random_state=42, shuffle=True)  

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")

Train size: 2645, Validation size: 1134

In [44]:
train_df["split"] = "train"
val_df["split"] = "val"

In [45]:
attributes_token = [item for sublist in train_df["attributes"] for item in sublist]

In [46]:
count_attributes = Counter(attributes_token)

In [47]:
most_common_attributes = count_attributes.most_common(50)

In [48]:
attributes = []

for i in range(len(most_common_attributes)):
    attributes.append(most_common_attributes[i][0])

In [49]:
attributes

['limits',
 'changes',
 'atelectasis',
 'disease',
 'opacities',
 'opacity',
 'calcifications',
 'granulomas',
 'tip',
 'midline',
 'clips',
 'effusion',
 'sternotomy',
 'density',
 'tortuosity',
 'size',
 'markings',
 'lymph',
 'appearance',
 'calcification',
 'effusions',
 'deformity',
 'elevation',
 'prominence',
 'borderline',
 'xxxx',
 'edema',
 'osteophytes',
 'eventration',
 'nodules',
 'scoliosis',
 'congestion',
 'hyperinflation',
 'pneumonia',
 'curvature',
 'enlargement',
 'consolidation',
 'densities',
 'lucency',
 'copd',
 'hyperexpansion',
 'fusion',
 'engorgement',
 'pneumothorax',
 'deformities',
 'aeration',
 'process',
 'cabg',
 'age',
 'indeterminate']

In [50]:
def split_into_sentences(text):
    # Tokenize the text into sentences
    doc = nlp(text)

    # Extract sentences
    sentences = [sent.text for sent in doc.sents]

    return sentences

In [51]:
train_df["sentences"] = train_df["text"].apply(split_into_sentences)

In [52]:
val_df["sentences"] = val_df["text"].apply(split_into_sentences)

In [53]:
from collections import Counter

from rich import print

for attribute in attributes:
    # Identify sentences that contain the attribute from the "sentences" column
    sentences_with_attributes = []

    for sentence_list in train_df["sentences"]:
        if sentence_list:
            for sentence in sentence_list:
                if attribute in sentence:
                    sentences_with_attributes.append(sentence)

    # Count the frequency of each sentence containing the attribute
    sentence_counter = Counter(sentences_with_attributes)

    # Get the 200 most frequent sentences
    most_common_sentences = [
        sentence for sentence, _ in sentence_counter.most_common(200)
    ]

    # Print the attribute and the two most common sentences for debugging
    print(f"Attribute: {attribute}")
    print(sentence_counter.most_common(5))

Attribute: limits

[
    ('heart size within normal limits .', 70),
    ('the heart size and pulmonary vascularity appear within normal limits .', 52),
    ('cardiac and mediastinal contours are within normal limits .', 48),
    ('mediastinal contour within normal limits .', 40),
    ('pulmonary vascularity is within normal limits .', 39)
]

Attribute: changes

[
    ('mild degenerative changes of the thoracic spine .', 16),
    ('degenerative changes in the thoracic spine .', 16),
    ('degenerative changes in the spine .', 15),
    ('there are atherosclerotic changes of the aorta .', 14),
    ('degenerative changes are present in the spine .', 11)
]

Attribute: atelectasis

...

[
    ('there has been interval cabg .', 2),
    ('there are postoperative changes of sternotomy and cabg .', 1),
    ('there are changes of xxxx sternotomy and cabg .', 1),
    ('there are finding status post sternotomy and cabg .', 1),
    (
        'heart size within normal limits , stable mediastinal and hilar contours , coronary artery stent artifact ,
xxxx xxxx and clips suggest cabg .',
        1
    )
]

Attribute: age

[
    ('2 images .', 13),
    ('osseous structures are within normal limits for patient age ..', 11),
    ('osseous structures are within normal limits for patient age .', 8),
    ('this is age - indeterminate .', 2),
    ('three images are available for review .', 2)
]

Attribute: indeterminate

[
    ('this is age - indeterminate .', 2),
    ('fractures of the posterior left 4th , 5th , and 6th ribs , age - indeterminate .', 2),
    ('this is age - indeterminate as no prior studies are available for comparison .', 2),
    (
        'there is a very mild anterior wedge deformity of a midthoracic vertebrae , possibly t7 , age - 
indeterminate .',
        1
    ),
    ('mild age indeterminate anterior wedging at the t11 level .', 1)
]

In [ ]:
from medclip import MedCLIPProcessor, MedCLIPTextModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

attribute_embeddings = []

processor = MedCLIPProcessor()
model = MedCLIPTextModel()
model.cuda()

for attribute in attributes:
    # Identify sentences that contain the attribute from the "sentences" column
    sentences_with_attributes = []

    for sentence_list in train_df["sentences"]:
        if sentence_list:
            for sentence in sentence_list:
                if attribute in sentence:
                    sentences_with_attributes.append(sentence)

    # Count the frequency of each sentence containing the attribute
    sentence_counter = Counter(sentences_with_attributes)

    # Get the 200 most frequent sentences
    most_common_sentences = [
        sentence for sentence, _ in sentence_counter.most_common(200)
    ]

    with torch.inference_mode():
        if most_common_sentences:
            # Compute sentence embeddings
            most_common_sentences = processor(
                text=most_common_sentences, return_tensors="pt", padding=True
            ).to("cuda")
            sentence_embeddings = model.forward(
                most_common_sentences["input_ids"],
                most_common_sentences["attention_mask"],
            )

            # Average the embeddings
            average_embedding = sentence_embeddings.mean(axis=0)
            average_embedding = average_embedding.cpu().detach().numpy()
            attribute_embeddings.append(average_embedding)
        else:
            print(f"No sentences found for attribute: {attribute}")

attribute_embeddings = np.stack(attribute_embeddings)
attribute_embeddings = torch.tensor(attribute_embeddings).to(device)
print(attribute_embeddings.shape)
attribute_embeddings = dict(zip(attributes, attribute_embeddings))

# Output the averaged embeddings
for attribute, embedding in attribute_embeddings.items():
    print(f"Shape: {embedding.shape}")
    print(f"Attribute: {attribute}")
    print(f"Embedding: {embedding[:10]}, dtype: {embedding.dtype}")

In [ ]:
# Save embeddings
torch.save(attribute_embeddings, f"{DATA_DIR}/attr_embs.pth")

In [55]:
train_df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[fibrosis, markings]",train,[interstitial markings are diffusely prominent...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[mastectomy, post]",train,"[status post left mastectomy ., heart size nor..."
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[air, atelectasis, calcification, density, opa...",train,[heart size and pulmonary vascularity appear w...
...,...,...,...,...,...,...,...,...,...,...,...,...
2653,2653,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[20, 20, 106, 194], [130, 21, 212, 202], [89,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the lungs are clear . the heart and pulmonary ...,[],train,"[the lungs are clear ., the heart and pulmonar..."
2654,2654,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 14, 103, 188], [129, 12, 216, 181], [86,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"heart size is enlarged , pulmonary vascularity...","[atelectasis, disease, effusion, limits]",train,"[heart size is enlarged , pulmonary vascularit..."
2655,2655,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[21, 10, 94, 170], [117, 10, 196, 190], [86, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart and lungs have xxxx xxxx in the inte...,[],train,[the heart and lungs have xxxx xxxx in the int...
2656,2656,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[32, 13, 107, 164], [131, 16, 200, 175], [90,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart size and cardiomediastinal silhouett...,[limits],train,[the heart size and cardiomediastinal silhouet...


In [54]:
regions = np.stack(train_df["region_coord"].tolist())

In [56]:
from tqdm.auto import tqdm

count = 0

for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    regions = row["region_coord"]
    shapes = [region.shape for region in regions]

    if len(set(shapes)) > 1:
        print(f"Count: {count}, Inconsistent shapes found in row {idx}: {shapes}")
        count += 1
        continue  # or handle the inconsistency

    regions = np.stack(regions)
    # Proceed with your processing

100%|██████████| 2645/2645 [00:01<00:00, 2352.83it/s]


In [57]:
train_df = train_df[
    train_df["region_coord"].apply(lambda x: not any(len(mask) == 0 for mask in x))
]
val_df = val_df[
    val_df["region_coord"].apply(lambda x: not any(len(mask) == 0 for mask in x))
]

In [58]:
from tqdm.auto import tqdm

count = 0

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):
    regions = row["region_coord"]
    shapes = [region.shape for region in regions]

    if len(set(shapes)) > 1:
        print(f"Count: {count}, Inconsistent shapes found in row {idx}: {shapes}")
        count += 1
        continue  # or handle the inconsistency

    regions = np.stack(regions)
    # Proceed with your processing

100%|██████████| 1134/1134 [00:00<00:00, 2372.37it/s]


In [59]:
frames = [train_df, val_df]

In [60]:
df = pd.concat(frames)

In [61]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[fibrosis, markings]",train,[interstitial markings are diffusely prominent...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[29, 15, 97, 180], [120, 12, 199, 200], [90, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[mastectomy, post]",train,"[status post left mastectomy ., heart size nor..."
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[air, atelectasis, calcification, density, opa...",train,[heart size and pulmonary vascularity appear w...
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[atherosclerosis, disease, tortuosity]",val,[cardiomediastinal silhouette demonstrates nor...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[body, midline, radiopaque]",val,"[normal heart and mediastinum ., clear lungs ...."


In [62]:
from tqdm.auto import tqdm

count = 0

for idx, row in tqdm(df.iterrows(), total=len(df)):
    regions = row["region_coord"]
    shapes = [region.shape for region in regions]

    if len(set(shapes)) > 1:
        print(f"Count: {count}, Inconsistent shapes found in row {idx}: {shapes}")
        count += 1
        continue  # or handle the inconsistency

    regions = np.stack(regions)
    # Proceed with your processing

100%|██████████| 3779/3779 [00:01<00:00, 2943.60it/s]


### Filter out attibutes that are not in the list of attributes


In [64]:
attributes

['limits',
 'changes',
 'atelectasis',
 'disease',
 'opacities',
 'opacity',
 'calcifications',
 'granulomas',
 'tip',
 'midline',
 'clips',
 'effusion',
 'sternotomy',
 'density',
 'tortuosity',
 'size',
 'markings',
 'lymph',
 'appearance',
 'calcification',
 'effusions',
 'deformity',
 'elevation',
 'prominence',
 'borderline',
 'xxxx',
 'edema',
 'osteophytes',
 'eventration',
 'nodules',
 'scoliosis',
 'congestion',
 'hyperinflation',
 'pneumonia',
 'curvature',
 'enlargement',
 'consolidation',
 'densities',
 'lucency',
 'copd',
 'hyperexpansion',
 'fusion',
 'engorgement',
 'pneumothorax',
 'deformities',
 'aeration',
 'process',
 'cabg',
 'age',
 'indeterminate']

In [65]:
df["attributes"] = df["attributes"].apply(
    lambda x: [attr for attr in x if attr in attributes]
)

In [69]:
df = df[df["attributes"].apply(lambda x: len(x) > 0)]

In [70]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,[markings],train,[interstitial markings are diffusely prominent...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[atelectasis, calcification, density, opacity]",train,[heart size and pulmonary vascularity appear w...
5,5,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[16, 25, 102, 178], [126, 25, 203, 183], [85,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary xxxx and mediastinum are...","[changes, limits]",train,"[the heart , pulmonary xxxx and mediastinum ar..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[disease, tortuosity]",val,[cardiomediastinal silhouette demonstrates nor...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,[midline],val,"[normal heart and mediastinum ., clear lungs ...."


In [71]:
# Check if the text column have an empty array or string
df = df[df["text"].apply(lambda x: len(x) > 0)]

In [72]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,"[atelectasis, limits, opacity]",train,[there is xxxx increased opacity within the ri...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,[markings],train,[interstitial markings are diffusely prominent...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[atelectasis, calcification, density, opacity]",train,[heart size and pulmonary vascularity appear w...
5,5,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[16, 25, 102, 178], [126, 25, 203, 183], [85,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary xxxx and mediastinum are...","[changes, limits]",train,"[the heart , pulmonary xxxx and mediastinum ar..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[disease, tortuosity]",val,[cardiomediastinal silhouette demonstrates nor...
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[limits],val,"[lungs are clear bilaterally ., cardiomediasti..."
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,[midline],val,"[normal heart and mediastinum ., clear lungs ...."


In [73]:
df = df[
    [
        "image_id",
        "image_size",
        "image_filepath",
        "region_coord",
        "region_bbox",
        "region_labels",
        "num_regions",
        "text_filepath",
        "text",
        "sentences",
        "attributes",
        "split",
    ]
]

In [74]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,sentences,attributes,split
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[7, 16, 111, 175], [142, 14, 223, 159], [87, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,[there is xxxx increased opacity within the ri...,"[atelectasis, limits, opacity]",train
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 13, 92, 172], [123, 12, 208, 176], [72, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...,[there is xxxx increased opacity within the ri...,"[atelectasis, limits, opacity]",train
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27, 23, 100, 170], [122, 23, 197, 192], [88,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,[interstitial markings are diffusely prominent...,[markings],train
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[17, 26, 92, 174], [117, 21, 213, 180], [78, ...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,[heart size and pulmonary vascularity appear w...,"[atelectasis, calcification, density, opacity]",train
5,5,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[16, 25, 102, 178], [126, 25, 203, 183], [85,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary xxxx and mediastinum are...","[the heart , pulmonary xxxx and mediastinum ar...","[changes, limits]",train
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[26, 11, 107, 172], [132, 18, 203, 194], [84,...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,[cardiomediastinal silhouette demonstrates nor...,"[disease, tortuosity]",val
3793,3793,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[25, 10, 98, 199], [125, 8, 193, 210], [88, 1...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[lungs are clear bilaterally ., cardiomediasti...",[limits],val
3794,3794,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[24, 1, 93, 158], [124, 2, 196, 185], [85, 76...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[lungs are clear bilaterally ., cardiomediasti...",[limits],val
3795,3795,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[31, 9, 99, 160], [123, 13, 199, 188], [88, 9...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[normal heart and mediastinum ., clear lungs ....",[midline],val


In [75]:
df.to_feather(f"{DATA_DIR}/annotations.feather")